In [3]:
import pyspark
spark = pyspark.sql.SparkSession(pyspark.SparkContext())
spark

In [5]:
import pandas as pd

# Convert excel to csv
def excel_to_csv(name):
    pd.read_excel(name + '.xlsx').to_csv(name + '.csv', sep=',', index=False)
    
excel_to_csv('Athletes')
excel_to_csv('Coaches')
excel_to_csv('EntriesGender')
excel_to_csv('Medals')
excel_to_csv('Teams')

c:\program files\python38\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [8]:
athletes = pd.read_csv('Athletes.csv')
coaches = pd.read_csv('Coaches.csv')
entries_gender = pd.read_csv('EntriesGender.csv')
medals = pd.read_csv('Medals.csv')
teams = pd.read_csv('Teams.csv')

athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [9]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [10]:
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
2,3,Japan,27,14,17,58,5
3,4,Great Britain,22,21,22,65,4
4,5,ROC,20,28,23,71,3


In [11]:
# Plot a bar chart for Medals
medals.sort_values('Rank by Total', inplace=True)
medals.head()

,Rank,Team/NOC,Gold,Silver,Bronze,Total,Rank by Total
0,1,United States of America,39,41,33,113,1
1,2,People's Republic of China,38,32,18,88,2
4,5,ROC,20,28,23,71,3
3,4,Great Britain,22,21,22,65,4
2,3,Japan,27,14,17,58,5


In [15]:
import plotly.express as px
fig = px.bar(medals, x='Team/NOC', y=['Gold', 'Silver', 'Bronze'],
             color_discrete_sequence=['Gold', 'Silver', 'Brown'],
             title='Number of Medals per country')
fig.show()

In [18]:
coaches.head()

,Name,NOC,Discipline,Event
0,ABDELMAGID Wael,Egypt,Football,NaN
1,ABE Junya,Japan,Volleyball,NaN
2,ABE Katsuhiko,Japan,Basketball,NaN
3,ADAMA Cherif,Côte d'Ivoire,Football,NaN
4,AGEBA Yuya,Japan,Volleyball,NaN


In [20]:
spark.read.csv('Coaches.csv', header=True, inferSchema=True).createOrReplaceTempView('Coaches')
spark.sql('Select * from Coaches').show(5)

+---------------+-------------+----------+-----+
|           Name|          NOC|Discipline|Event|
+---------------+-------------+----------+-----+
|ABDELMAGID Wael|        Egypt|  Football| null|
|      ABE Junya|        Japan|Volleyball| null|
|  ABE Katsuhiko|        Japan|Basketball| null|
|   ADAMA Cherif|Côte d'Ivoire|  Football| null|
|     AGEBA Yuya|        Japan|Volleyball| null|
+---------------+-------------+----------+-----+
only showing top 5 rows



In [25]:
pd_coaches_query = spark.sql('Select NOC, Count(NOC) as NOC_Count from Coaches group by NOC order by NOC').toPandas()
pd_coaches_query.head(10)

,NOC,NOC_Count
0,Angola,1
1,Argentina,11
2,Australia,22
3,Austria,3
4,Bahrain,1
5,Belarus,1
6,Belgium,5
7,Brazil,7
8,Cambodia,1
9,Canada,16


In [29]:
fig = px.bar(pd_coaches_query, x='NOC', y='NOC_Count', color='NOC_Count', title='Number of Coaches by Country')
fig.show()

In [30]:
athletes.head()

,Name,NOC,Discipline
0,AALERUD Katrine,Norway,Cycling Road
1,ABAD Nestor,Spain,Artistic Gymnastics
2,ABAGNALE Giovanni,Italy,Rowing
3,ABALDE Alberto,Spain,Basketball
4,ABALDE Tamara,Spain,Basketball


In [33]:
spark.read.csv('Athletes.csv', header=True, inferSchema=True).createOrReplaceTempView('Athletes')
spark.sql('select * from athletes order by NOC, Discipline').show(20)

+--------------------+-----------+-------------------+
|                Name|        NOC|         Discipline|
+--------------------+-----------+-------------------+
|       YOUSOFI Kimia|Afghanistan|          Athletics|
|NOOR ZAHI Sha Mah...|Afghanistan|          Athletics|
|        YOVARI Mahdi|Afghanistan|           Shooting|
|        ANWARI Fahim|Afghanistan|           Swimming|
|     MANSOURI Farzad|Afghanistan|          Taekwondo|
|       PETROV Matvei|    Albania|Artistic Gymnastics|
|       SMAJLAJ Izmir|    Albania|          Athletics|
|      CULLHAJ Indrit|    Albania|               Judo|
|     DELILAJ Manuela|    Albania|           Shooting|
|         KADIU Kledi|    Albania|           Swimming|
|       MERIZAJ Nikol|    Albania|           Swimming|
|        CALJA Briken|    Albania|      Weightlifting|
|      QERIMAJ Erkand|    Albania|      Weightlifting|
|      HETHAT Yassine|    Algeria|          Athletics|
|TRIKI Yasser Mohamed|    Algeria|          Athletics|
| LAHOULOU

In [35]:
pd_athletes_query = spark.sql('''select first(NOC) as NOC, first(Discipline) as Discipline, count(*) as Count from athletes group by NOC, Discipline order by NOC, Discipline''').toPandas()

pd_athletes_query.head(10)

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
1,Afghanistan,Shooting,1
2,Afghanistan,Swimming,1
3,Afghanistan,Taekwondo,1
4,Albania,Artistic Gymnastics,1
5,Albania,Athletics,1
6,Albania,Judo,1
7,Albania,Shooting,1
8,Albania,Swimming,2
9,Albania,Weightlifting,2


In [37]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes from each country by Discipline')
fig.show()

In [38]:
athletes['Discipline'].value_counts().head()

Athletics    2068
Swimming      743
Football      567
Rowing        496
Hockey        406
Name: Discipline, dtype: int64

In [41]:
pd_athletes_query = pd_athletes_query.loc[pd_athletes_query['Discipline'].isin(['Athletics', 'Swimming','Football'])]
pd_athletes_query.head()

,NOC,Discipline,Count
0,Afghanistan,Athletics,2
2,Afghanistan,Swimming,1
5,Albania,Athletics,1
8,Albania,Swimming,2
10,Algeria,Athletics,8


In [42]:
pd_athletes_query.Discipline.value_counts()

Athletics    195
Swimming     181
Football      24
Name: Discipline, dtype: int64

In [43]:
fig = px.bar(pd_athletes_query, x='NOC', y='Count', color='Discipline', title='Number of Athletes from each country by Discipline')
fig.show()

In [44]:
entries_gender.head()

,Discipline,Female,Male,Total
0,3x3 Basketball,32,32,64
1,Archery,64,64,128
2,Artistic Gymnastics,98,98,196
3,Artistic Swimming,105,0,105
4,Athletics,969,1072,2041


In [45]:
len(entries_gender)

46

In [47]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Shape of chart
specs = [[{'type': 'domain'}, {'type': 'domain'}]]*23
specs

[[{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}],
 [{'type': 'domain'}, {'type': 'domain'}]]

In [65]:
# Plot pie chart
fig = make_subplots(rows=23, cols=2,
                   subplot_titles=entries_gender['Discipline'], specs=specs)

sub_figs = []
for index, row in entries_gender.iterrows():
    sub_fig = go.Pie(labels=['Female', 'Male'], values=[row['Female'], 
                                                        row['Male']])
    sub_figs.append(sub_fig)

k=0
for i in range(1,24):
    for j in range(1,3):
        fig.add_trace(sub_figs[k], i, j)
        k += 1
        
fig.update_layout(showlegend=False, height=10000, width=800,  title_text='Distribution of Gender amongst each game')
fig.update_traces(textposition='inside', textinfo='label+percent', hoverinfo='label+value+percent')

fig = go.Figure(fig)
fig.show()

In [68]:
teams.head()

,Name,Discipline,NOC,Event
0,Belgium,3x3 Basketball,Belgium,Men
1,China,3x3 Basketball,People's Republic of China,Men
2,China,3x3 Basketball,People's Republic of China,Women
3,France,3x3 Basketball,France,Women
4,Italy,3x3 Basketball,Italy,Women


In [69]:
spark.read.csv('Teams.csv', header=True, inferSchema=True).createOrReplaceTempView('Teams')
spark.sql('select * from teams order by Discipline').show(5)

+-------+--------------+--------------------+-----+
|   Name|    Discipline|                 NOC|Event|
+-------+--------------+--------------------+-----+
|Belgium|3x3 Basketball|             Belgium|  Men|
|  China|3x3 Basketball|People's Republic...|  Men|
|  China|3x3 Basketball|People's Republic...|Women|
| France|3x3 Basketball|              France|Women|
|  Italy|3x3 Basketball|               Italy|Women|
+-------+--------------+--------------------+-----+
only showing top 5 rows



In [98]:
pd_teams_query = spark.sql('''select Discipline, count(distinct Name) as number_of_country from Teams group by Discipline order by number_of_country ''').toPandas()

pd_teams_query.head(10)

,Discipline,number_of_country
0,Baseball/Softball,9
1,3x3 Basketball,13
2,Rhythmic Gymnastics,14
3,Hockey,14
4,Rugby Sevens,16
5,Water Polo,16
6,Artistic Gymnastics,17
7,Basketball,18
8,Handball,18
9,Triathlon,18


In [99]:
pd_teams_query.shape

(20, 2)

In [100]:
fig = px.bar(pd_teams_query, x='number_of_country', y='Discipline', color='number_of_country',
             title='Number of country by Discipline', height=700)
fig.show()